In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import numpy as np
from numpy import random
database = 'database.sqlite'
conn = sqlite3.connect('../database/database.sqlite')

In [14]:
sql = """select league_id,season,stage,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,B365H,B365D,B365A from Match where league_id=21518 or league_id= 1729 or league_id=4769 or league_id=10257"""                  

data = pd.read_sql(sql,conn)
data

,league_id,season,stage,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,B365H,B365D,B365A
0,1729,2008/2009,1,10260,10261,1,1,1.29,5.50,11.00
1,1729,2008/2009,1,9825,8659,1,0,1.20,6.50,15.00
2,1729,2008/2009,1,8472,8650,0,1,5.50,3.60,1.67
3,1729,2008/2009,1,8654,8528,2,1,1.91,3.40,4.20
4,1729,2008/2009,1,10252,8456,4,2,1.91,3.40,4.33
5,1729,2008/2009,1,8668,8655,2,3,2.00,3.30,4.00
6,1729,2008/2009,1,8549,8586,2,1,3.20,3.40,2.25
7,1729,2008/2009,1,8559,10194,3,1,1.83,3.50,4.50
8,1729,2008/2009,1,8667,9879,2,1,2.60,3.20,2.80
9,1729,2008/2009,1,8455,8462,4,0,1.33,5.00,10.00


In [15]:
#Converting season values

def change_season(match):
    if match['season'] =='2008/2009':
        return 0
    elif match['season'] =='2009/2010':
        return 1
    elif match['season'] =='2010/2011':
        return 2
    elif match['season'] =='2011/2012':
        return 3
    elif match['season'] =='2012/2013':
        return 4
    elif match['season'] =='2013/2014':
        return 5
    elif match['season'] =='2014/2015':
        return 6
    else:
        return 7
    
data['season']=data.apply(change_season, axis=1) 

#Classifing Match output
def determine_home_result(match):
    if match['home_team_goal'] > match['away_team_goal']:
        return 1
    elif match['home_team_goal'] < match['away_team_goal']:
        return 2
    else:
        return 0
    
    
data['Output']=data.apply(determine_home_result, axis=1)

data= data.dropna()



In [16]:
#função SVM

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import log_loss


train = data[data['season']<=5]
val = data[data['season']==6]
test = data[data['season']==7]

ft=['league_id','season','stage','home_team_api_id','away_team_api_id','B365H','B365D','B365A']
out=['Output']
def SVM_validation(kernel,C,train,val,ft,out):
    best=[]
    sc_max=-10000
    print kernel
    print C
    print ft
    print out
    for kernel_type in kernel:
        print kernel_type
        for cv in C:
            print cv
            clf = SVC(C=cv,kernel=kernel_type)
            clf.fit(train[ft],train[out])
            score = log_loss(val[out], clf.predict(val[ft]))
            if score>sc_max:
                best=clf
                sc_max=score
    return best

c = range(1,2)
kernel=['poly', 'rbf', 'sigmoid']
#svm = SVM_validation (kernel,c,train,val,ft,out)


In [21]:
#Função logreg
from sklearn import linear_model
from sklearn.metrics import f1_score

def logreg_validation(C,solver,train,val,ft,out):
    sc_max = -10000
    best=[]
    for s in solver:
        for cv in C:
            print s,cv
            lg = linear_model.LogisticRegression(C=cv,solver=s,n_jobs=-1,max_iter=100000)
            lg.fit(train[ft],train[out])
            score = f1_score(val[out], lg.predict(val[ft]),average='weighted')
            if score>sc_max:
                best=lg
                sc_max=score
    return best

C= [0.001, 0.01, 0.1, 1, 10, 100, 1000]
solver=['newton-cg', 'lbfgs', 'liblinear', 'sag']
logreg=logreg_validation(C,solver,train,val,ft,out)
print logreg


newton-cg 0.001


/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/sklearn/

newton-cg 0.01


/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/sklearn/

newton-cg 0.1


...

newton-cg 100


/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/op

newton-cg 1000


/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/scipy/op

lbfgs 0.001
lbfgs 0.01
lbfgs 0.1
lbfgs 1
lbfgs 10
lbfgs 100
lbfgs 1000
liblinear 0.001
liblinear 0.01
liblinear 0.1
liblinear 1
liblinear 10
liblinear 100
liblinear 1000
sag 0.001
sag 0.01
sag 0.1
sag 1
sag 10
sag 100
sag 1000
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='ovr',
          n_jobs=-1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)


In [24]:
print f1_score(test[out],logreg.predict(test[ft]),average='weighted')
print f1_score(test[out],logreg.predict(test[ft]),average='macro')
print f1_score(test[out],logreg.predict(test[ft]),average='micro')


0.423700119474
0.372542829717
0.504279131007


In [34]:
#Função RandomForest
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

def rf_validation(n_estimators,train,val,ft,out):
    sc_max = -10000
    best=[]
    for n in n_estimators:
        rf = RandomForestClassifier(n_estimators=n,n_jobs=-1)
        rf.fit(train[ft],train[out])
        score = f1_score(val[out], rf.predict(val[ft]),average='weighted')
        if score>sc_max:
            best=rf
            sc_max=score
    return best

n = range(100,1000,50)

rf= rf_validation(n,train,val,ft,out)

print f1_score(test[out],rf.predict(test[ft]),average='weighted')
print f1_score(test[out],rf.predict(test[ft]),average='macro')
print f1_score(test[out],rf.predict(test[ft]),average='micro')

#Função Naive Bayes
from sklearn.naive_bayes import GaussianNB
def nb_validation(train,val,ft,out):
    nb = GaussianNB()
    nb.fit(train[ft],train[out])
    return nb

nb= nb_validation(train,val,ft,out)
print "Gaussian NB"
print f1_score(test[out],nb.predict(test[ft]),average='weighted')
print f1_score(test[out],nb.predict(test[ft]),average='macro')
print f1_score(test[out],nb.predict(test[ft]),average='micro')
#Gradient Boosting
def gb_validation(learning_rate,n_estimators,train,val,ft,out):
    sc_max = -10000
    best=[]
    for n in n_estimators:
        for lr in learning_rate:
            gb = GradientBoostingClassifier(learning_rate=lr,n_estimators=n)
            gb.fit(train[ft],train[out])
            score = f1_score(val[out], gb.predict(val[ft]),average='weighted')
            if score>sc_max:
                best=gb
                sc_max=score
    return best

lr= [0.0001, 0.001, 0.01, 0.1]
gb= gb_validation(lr,n,train,val,ft,out)

print "GradientBoosting NB"
print f1_score(test[out],gb.predict(test[ft]),average='weighted')
print f1_score(test[out],gb.predict(test[ft]),average='macro')
print f1_score(test[out],gb.predict(test[ft]),average='micro')


/home/felipef/anaconda2/envs/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


0.439711366681
0.400354070005
0.479262672811
Gaussian NB
0.405746672326
0.362069633601
0.470704410797
GradientBoosting NB
0.463657947679
0.426453086191
0.499012508229


In [67]:
#GaussianProcess
ynaive = [train['Output'].mode()[0]]*len(test)
naive = f1_score(test[out],ynaive,average='weighted')
#print "LogReg
print "\t\tF1Score\t","      F1 macro\t","    F1 micro"
print "Naive      ", naive,f1_score(test[out],ynaive,average='macro'),f1_score(test[out],ynaive,average='micro')
print "LogReg     ", f1_score(test[out],logreg.predict(test[ft]),average='weighted'),f1_score(test[out],logreg.predict(test[ft]),average='macro'),f1_score(test[out],logreg.predict(test[ft]),average='micro')
print "Naive Bayes", f1_score(test[out],nb.predict(test[ft]),average='weighted'),f1_score(test[out],nb.predict(test[ft]),average='macro'),f1_score(test[out],nb.predict(test[ft]),average='micro')
print "Random F.  ", f1_score(test[out],rf.predict(test[ft]),average='weighted'),f1_score(test[out],rf.predict(test[ft]),average='macro'),f1_score(test[out],rf.predict(test[ft]),average='micro')
print "G. Boost.  ", f1_score(test[out],gb.predict(test[ft]),average='weighted'),f1_score(test[out],gb.predict(test[ft]),average='macro'),f1_score(test[out],gb.predict(test[ft]),average='micro')



		F1Score	      F1 macro	    F1 micro
Naive       0.273428099737 0.205104831358 0.444371296906
LogReg      0.423700119474 0.372542829717 0.504279131007
Naive Bayes 0.405746672326 0.362069633601 0.470704410797
Random F.   0.439711366681 0.400354070005 0.479262672811
G. Boost.   0.463657947679 0.426453086191 0.499012508229


In [69]:
def IDtoStr(league_id):
    league_id = int(league_id)
    if league_id == 1729 : 
        return "Liga Inglesa"
    elif league_id == 4769 : 
        return "Liga Francesa"
    elif league_id == 7809 : 
        return "Liga Alema"
    elif league_id == 10257 : 
        return "Liga Italiana"
    elif league_id == 21518 : 
        return "Liga Espanhola"
    else:
        return "Liga Portuguesa"
    



In [71]:
featured = pd.read_csv('dados2.csv')
featured['Output']=featured.apply(determine_home_result, axis=1)
featured['season']=featured.apply(change_season, axis=1) 

featured= featured.dropna()

import pandasql as pdsql
pysql = lambda q: pdsql.sqldf(q, globals())

#data = pysql("Select * from data where league_id=21518 or league_id= 1729 or league_id=4769 or league_id=10257")

featured = pysql("Select * from featured where league_id=21518 or league_id= 1729 or league_id=4769 or league_id=10257")


from sklearn.feature_selection import chi2

ft=[u'hg_V', u'hg_D', u'hg_E', u'hg_GF', u'hg_AVG_GF', u'hg_GS',
       u'hg_AVG_GS', u'hg_J1GS', u'hg_J1GF', u'hg_BTTS', u'hg_Over2',
       u'hg_Over3', u'hg_Over4', u'hg_WinningCleanSheet', u'hg_LosingFS',
       u'hg_FailedScore', u'hg_CleanSheet', u'hg_AVG_GOALS', u'hgc_V',
       u'hgc_D', u'hgc_E', u'hgc_GF', u'hgc_AVG_GF', u'hgc_GS',
       u'hgc_AVG_GS', u'hgc_J1GS', u'hgc_J1GF', u'hgc_BTTS', u'hgc_Over2',
       u'hgc_Over3', u'hgc_Over4', u'hgc_WinningCleanSheet',
       u'hgc_LosingFS', u'hgc_FailedScore', u'hgc_CleanSheet',
       u'hgc_AVG_GOALS', u'hl_V', u'hl_D', u'hl_E', u'hl_GF', u'hl_AVG_GF',
       u'hl_GS', u'hl_AVG_GS', u'hl_J1GS', u'hl_J1GF', u'hl_BTTS',
       u'hl_Over2', u'hl_Over3', u'hl_Over4', u'hl_WinningCleanSheet',
       u'hl_LosingFS', u'hl_FailedScore', u'hl_CleanSheet',
       u'hl_AVG_GOALS', u'hlc_V', u'hlc_D', u'hlc_E', u'hlc_GF',
       u'hlc_AVG_GF', u'hlc_GS', u'hlc_AVG_GS', u'hlc_J1GS', u'hlc_J1GF',
       u'hlc_BTTS', u'hlc_Over2', u'hlc_Over3', u'hlc_Over4',
       u'hlc_WinningCleanSheet', u'hlc_LosingFS', u'hlc_FailedScore',
       u'hlc_CleanSheet', u'hlc_AVG_GOALS', u'ag_V', u'ag_D', u'ag_E',
       u'ag_GF', u'ag_AVG_GF', u'ag_GS', u'ag_AVG_GS', u'ag_J1GS',
       u'ag_J1GF', u'ag_BTTS', u'ag_Over2', u'ag_Over3', u'ag_Over4',
       u'ag_WinningCleanSheet', u'ag_LosingFS', u'ag_FailedScore',
       u'ag_CleanSheet', u'ag_AVG_GOALS', u'agc_V', u'agc_D', u'agc_E',
       u'agc_GF', u'agc_AVG_GF', u'agc_GS', u'agc_AVG_GS', u'agc_J1GS',
       u'agc_J1GF', u'agc_BTTS', u'agc_Over2', u'agc_Over3', u'agc_Over4',
       u'agc_WinningCleanSheet', u'agc_LosingFS', u'agc_FailedScore',
       u'agc_CleanSheet', u'agc_AVG_GOALS', u'al_V', u'al_D', u'al_E',
       u'al_GF', u'al_AVG_GF', u'al_GS', u'al_AVG_GS', u'al_J1GS',
       u'al_J1GF', u'al_BTTS', u'al_Over2', u'al_Over3', u'al_Over4',
       u'al_WinningCleanSheet', u'al_LosingFS', u'al_FailedScore',
       u'al_CleanSheet', u'al_AVG_GOALS', u'alc_V', u'alc_D', u'alc_E',
       u'alc_GF', u'alc_AVG_GF', u'alc_GS', u'alc_AVG_GS', u'alc_J1GS',
       u'alc_J1GF', u'alc_BTTS', u'alc_Over2', u'alc_Over3', u'alc_Over4',
       u'alc_WinningCleanSheet', u'alc_LosingFS', u'alc_FailedScore',
       u'alc_CleanSheet', u'alc_AVG_GOALS', u'ga_rank_def', u'ga_rank_atk',
       u'gh_rank_def', u'gh_rank_atk', u'la_rank_def', u'la_rank_atk',
       u'lh_rank_def', u'lh_rank_atk', u'a_date', u'h_date', u'stage','B365H','B365D','B365A']


ft_rank,pvalue = chi2(ft_train[ft],ft_train[out])
ft_rank

array([  5.24647806e+02,   5.19598396e+02,   9.52323327e+00,
         1.26433404e+03,   6.59130801e+01,   5.28868962e+02,
         2.05073410e+01,   8.96124627e+01,   1.04726740e+02,
         4.89243058e+00,   2.56966163e+01,   7.72855776e+01,
         8.58746579e+01,   2.63043078e+02,   3.94795584e+02,
         3.50156779e+02,   1.38041559e+02,   7.03810182e+00,
         2.13813299e+02,   2.81199015e+02,   2.42380619e+01,
         6.79514134e+02,   7.24329400e+01,   2.03094393e+02,
         1.55640608e+01,   5.17560604e+01,   3.38012144e+01,
         1.53326883e+00,   2.65480515e+01,   7.46690427e+01,
         9.58395302e+01,   1.17771851e+02,   1.84601365e+02,
         1.64124599e+02,   6.16613863e+01,   1.61085149e+01,
         1.25465024e+02,   9.38709509e+01,   1.93580454e+00,
         2.58334732e+02,   5.11931009e+01,   9.89158466e+01,
         2.02527607e+01,   2.05058732e+01,   2.48734744e+01,
         1.35131653e-01,   4.48562719e+00,   1.19251066e+01,
         1.27993827e+01,

In [82]:
rank={}
i=0
for v in ft_rank:
    if v>=100:
        rank[i]=v
    i=i+1

import operator

sorted_x = sorted(rank.items(), key=operator.itemgetter(1),reverse=True)
sorted_x

[(157, 2227.646296547724),
 (75, 1612.8496062112019),
 (3, 1264.3340444561754),
 (155, 993.49574355211303),
 (147, 823.33565245790498),
 (72, 720.96977349543215),
 (21, 679.51413355271461),
 (145, 670.65821320455086),
 (93, 648.00782463013081),
 (146, 572.27667481086803),
 (5, 528.86896196575435),
 (0, 524.64780622404032),
 (1, 519.59839616371482),
 (73, 446.1621567911011),
 (144, 427.57068859312915),
 (85, 411.06906945344826),
 (77, 399.42167264736304),
 (14, 394.79558447609327),
 (90, 388.63840234894411),
 (15, 350.15677875820347),
 (86, 296.83536099061507),
 (111, 291.86754653224205),
 (19, 281.19901470504828),
 (13, 263.04307828995445),
 (39, 258.33473207557358),
 (88, 255.16922848404576),
 (87, 247.72954945150201),
 (151, 245.50248222661128),
 (103, 235.06923447387837),
 (18, 213.81329894808033),
 (95, 204.62033554437451),
 (23, 203.09439292065861),
 (150, 202.76902717394375),
 (91, 193.22973879331408),
 (32, 184.60136514291884),
 (33, 164.12459900736957),
 (149, 159.6363960223620

In [95]:
selected_ft=[]
for key,value in sorted_x:
    selected_ft.append(ft[key])
    print ft[key],value,pvalue[key]

B365A 2227.64629655 0.0
ag_GF 1612.84960621 0.0
hg_GF 1264.33404446 2.84021100025e-275
B365H 993.495743552 1.84136714748e-216
gh_rank_atk 823.335652458 1.64034308312e-179
ag_V 720.969773495 2.77589398576e-157
hgc_GF 679.514133553 2.78856461005e-148
ga_rank_atk 670.658213205 2.3357124792e-146
agc_GF 648.00782463 1.93592606217e-141
gh_rank_def 572.276674811 5.39136833838e-125
hg_GS 528.868961966 1.43735513281e-115
hg_V 524.647806224 1.18625049628e-114
hg_D 519.598396164 1.48129606843e-113
ag_D 446.162156791 1.30953959976e-97
ga_rank_def 427.570688593 1.42627956602e-93
ag_WinningCleanSheet 411.069069453 5.46368594088e-90
ag_GS 399.421672647 1.84793124382e-87
hg_LosingFS 394.795584476 1.86736016948e-86
agc_V 388.638402349 4.05735681179e-85
hg_FailedScore 350.156778758 9.21343533663e-77
ag_LosingFS 296.835360991 3.49156670924e-65
al_GF 291.867546532 4.18569469047e-64
hgc_D 281.199014705 8.67780323492e-62
hg_WinningCleanSheet 263.04307829 7.60188504426e-58
hl_GF 258.334732076 8.00434294795e-

In [96]:
selected_ft
C= [0.001, 0.01, 0.1, 1, 10, 100, 1000]
solver=['newton-cg', 'lbfgs', 'liblinear', 'sag']
ft_train = featured[featured['season']<=5]
ft_val = featured[featured['season']==6]
ft_test = featured[featured['season']==7]
logreg_select=logreg_validation(C,solver,ft_train,ft_val,selected_ft,out)
n = range(100,1000,50)
rf_select= rf_validation(n,ft_train,ft_val,selected_ft,out)
lr= [0.0001, 0.001, 0.01, 0.1]
gb_select= gb_validation(lr,n,train,val,ft,out)
nb_select= nb_validation(train,val,ft,out)

['B365A',
 u'ag_GF',
 u'hg_GF',
 'B365H',
 u'gh_rank_atk',
 u'ag_V',
 u'hgc_GF',
 u'ga_rank_atk',
 u'agc_GF',
 u'gh_rank_def',
 u'hg_GS',
 u'hg_V',
 u'hg_D',
 u'ag_D',
 u'ga_rank_def',
 u'ag_WinningCleanSheet',
 u'ag_GS',
 u'hg_LosingFS',
 u'agc_V',
 u'hg_FailedScore',
 u'ag_LosingFS',
 u'al_GF',
 u'hgc_D',
 u'hg_WinningCleanSheet',
 u'hl_GF',
 u'ag_CleanSheet',
 u'ag_FailedScore',
 u'lh_rank_atk',
 u'agc_WinningCleanSheet',
 u'hgc_V',
 u'agc_GS',
 u'hgc_GS',
 u'lh_rank_def',
 u'agc_D',
 u'hgc_LosingFS',
 u'hgc_FailedScore',
 u'la_rank_atk',
 u'al_V',
 u'agc_CleanSheet',
 u'la_rank_def',
 u'ag_J1GF',
 u'agc_LosingFS',
 u'hg_CleanSheet',
 u'hlc_GF',
 u'hl_V',
 u'al_GS',
 u'hgc_WinningCleanSheet',
 u'ag_Over4',
 u'agc_FailedScore',
 u'hg_J1GF',
 u'al_D',
 u'al_WinningCleanSheet']

In [ ]:
print "\t\tF1Score\t","      F1 macro\t","    F1 micro"
print "Naive      ", naive,f1_score(test[out],ynaive,average='macro'),f1_score(test[out],ynaive,average='micro')
print "LogReg     ", f1_score(test[out],logreg.predict(test[ft]),average='weighted'),f1_score(test[out],logreg.predict(test[ft]),average='macro'),f1_score(test[out],logreg.predict(test[ft]),average='micro')
print "Naive Bayes", f1_score(test[out],nb.predict(test[ft]),average='weighted'),f1_score(test[out],nb.predict(test[ft]),average='macro'),f1_score(test[out],nb.predict(test[ft]),average='micro')
print "Random F.  ", f1_score(test[out],rf.predict(test[ft]),average='weighted'),f1_score(test[out],rf.predict(test[ft]),average='macro'),f1_score(test[out],rf.predict(test[ft]),average='micro')
print "G. Boost.  ", f1_score(test[out],gb.predict(test[ft]),average='weighted'),f1_score(test[out],gb.predict(test[ft]),average='macro'),f1_score(test[out],gb.predict(test[ft]),average='micro')

